In [13]:
!pip install -q numerapi pandas pyarrow matplotlib lightgbm scikit-learn cloudpickle==2.2.1 scipy==1.11.4

import os
import json

%matplotlib inline

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.0.2 requires torch>=1.11.0, which is not installed.
tensorboard 2.15.0 requires grpcio>=1.48.2, which is not installed.
tensorflow-macos 2.15.0 requires grpcio<2.0,>=1.24.3, which is not installed.
torchmetrics 0.11.4 requires torch>=1.8.1, which is not installed.


In [2]:
# Initialize NumerAPI - the official Python API client for Numerai
from numerapi import NumerAPI
napi = NumerAPI()

# list the datasets and available versions
all_datasets = napi.list_datasets()
dataset_versions = list(set(d.split('/')[0] for d in all_datasets))
print("Available versions:\n", dataset_versions)

# Set data version to one of the latest datasets
DATA_VERSION = "v5.0"

# Print all files available for download for our version
current_version_files = [f for f in all_datasets if f.startswith(DATA_VERSION)]
print("Available", DATA_VERSION, "files:\n", current_version_files)

Available versions:
 ['signals', 'v5.0', 'crypto']
Available v5.0 files:
 ['v5.0/features.json', 'v5.0/live.parquet', 'v5.0/live_benchmark_models.parquet', 'v5.0/live_example_preds.csv', 'v5.0/live_example_preds.parquet', 'v5.0/meta_model.parquet', 'v5.0/train.parquet', 'v5.0/train_benchmark_models.parquet', 'v5.0/validation.parquet', 'v5.0/validation_benchmark_models.parquet', 'v5.0/validation_example_preds.csv', 'v5.0/validation_example_preds.parquet']


In [3]:
DATA_VERSION = "v5.0"


data_dir = '../../data/'
train_path = os.path.join(data_dir, 'train.parquet')
validation_path = os.path.join(data_dir, 'validation.parquet')
live_path = os.path.join(data_dir, 'live.parquet')
metadata_path = os.path.join(data_dir, 'features.json')

# read the metadata and display
feature_metadata = json.load(open(metadata_path))
for metadata in feature_metadata:
  print(metadata, len(feature_metadata[metadata]))

feature_sets 17
targets 37


In [4]:
feature_sets = feature_metadata["feature_sets"]
for feature_set in ["small", "medium", "all"]:
  print(feature_set, len(feature_sets[feature_set]))

small 42
medium 705
all 2376


In [19]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


NumPy version: 2.0.1
Pandas version: 2.2.3


In [6]:


# Downgrade NumPy
!pip install numpy==1.21.6

# Optionally, reinstall Pandas
!pip install --upgrade pandas

  Using cached numpy-1.21.6-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
Using cached numpy-1.21.6-cp310-cp310-macosx_11_0_arm64.whl (12.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.0.2 requires torch>=1.11.0, which is not installed.
tensorboard 2.15.0 requires grpcio>=1.48.2, which is not installed.
tensorflow-macos 2.15.0 requires grpcio<2.0,>=1.24.3, which is not installed.
torchmetrics 0.11.4 requires torch>=1.8.1, which is not installed.
matplotlib 3.10.3 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 1.21.6 which is incompatible.
numerai-tools 0.2.7 requires numpy<2.0.0,>=1.26.4, but you ha

In [8]:
import pandas as pd

# Define our feature set
feature_set = feature_sets["medium"]
# use "medium" or "all" for better performance. Requires more RAM.
# features = feature_metadata["feature_sets"]["medium"]
# features = feature_metadata["feature_sets"]["all"]

# Download the training data - this will take a few minutes

# Load only the "medium" feature set to
# Use the "all" feature set to use all features
train = pd.read_parquet(
    train_path,
    columns=["era", "target"] + feature_set
)

# Downsample to every 4th era to reduce memory usage and speedup model training (suggested for Colab free tier)
# Comment out the line below to use all the data
train = train[train["era"].isin(train["era"].unique()[::4])]

In [11]:
# Plot the number of rows per era
train.groupby("era").size().plot(
    title="Number of rows per era",
    figsize=(5, 3),
    xlabel="Era"
)

<Axes: title={'center': 'Number of rows per era'}, xlabel='Era'>

In [12]:
# Plot density histogram of the target
train["target"].plot(
  kind="hist",
  title="Target",
  figsize=(5, 3),
  xlabel="Value",
  density=True,
  bins=50
)

<Axes: title={'center': 'Target'}, xlabel='Value', ylabel='Frequency'>

In [15]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
first_era = train[train["era"] == train["era"].unique()[0]]
last_era = train[train["era"] == train["era"].unique()[-1]]
last_era[feature_set[-1]].plot(
   title="5 equal bins",
   kind="hist",
   density=True,
   bins=50,
   ax=ax1
)
first_era[feature_set[-1]].plot(
   title="missing data",
   kind="hist",
   density=True,
   bins=50,
   ax=ax2
)

<Axes: title={'center': 'missing data'}, ylabel='Frequency'>

In [17]:
import lightgbm as lgb

#We've found the following "deep" parameters perform much better, but they require much more CPU and RAM

model = lgb.LGBMRegressor(
n_estimators=30_000,
learning_rate=0.001,
max_depth=10,
num_leaves=2**10,
colsample_bytree=0.1,
min_data_in_leaf=10000,
)

# This will take a few minutes 🍵
model.fit(
  train[feature_set],
  train["target"]
)

[LightGBM] [Warning] min_data_in_leaf is set=10000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10000
[LightGBM] [Warning] min_data_in_leaf is set=10000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008840 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3525
[LightGBM] [Info] Number of data points in the train set: 688184, number of used features: 705
[LightGBM] [Info] Start training from score 0.500008
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

LGBMRegressor(colsample_bytree=0.1, learning_rate=0.001, max_depth=10,
              min_data_in_leaf=10000, n_estimators=30000, num_leaves=1024)

In [18]:
# Load the validation data and filter for data_type == "validation"
validation = pd.read_parquet(
    validation_path,
    columns=["era", "data_type", "target"] + feature_set
)
validation = validation[validation["data_type"] == "validation"]
del validation["data_type"]

# Downsample to every 4th era to reduce memory usage and speedup evaluation (suggested for Colab free tier)
# Comment out the line below to use all the data (slower and higher memory usage, but more accurate evaluation)
validation = validation[validation["era"].isin(validation["era"].unique()[::4])]

# Eras are 1 week apart, but targets look 20 days (o 4 weeks/eras) into the future,
# so we need to "embargo" the first 4 eras following our last train era to avoid "data leakage"
last_train_era = int(train["era"].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(4)]]
validation = validation[~validation["era"].isin(eras_to_embargo)]

# Generate predictions against the out-of-sample validation features
# This will take a few minutes 🍵
validation["prediction"] = model.predict(validation[feature_set])
validation[["era", "prediction", "target"]]

[LightGBM] [Warning] min_data_in_leaf is set=10000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10000


,era,prediction,target
id,,,
n000c290e4364875,0579,0.483248,0.50
n002a15bc5575bbb,0579,0.508447,0.25
n00309caaa0f955e,0579,0.518505,0.75
n0039cbdcf835708,0579,0.492828,0.50
n004143458984f89,0579,0.501670,0.50
...,...,...,...
nfff25c0fac1ffd6,1151,0.506298,0.50
nfff53f275fc23e2,1151,0.493948,0.75
nfff68da2902848d,1151,0.507864,0.50


In [19]:
# install Numerai's open-source scoring tools
!pip install -q --no-deps numerai-tools

# import the 2 scoring functions
from numerai_tools.scoring import numerai_corr, correlation_contribution

# Download and join in the meta_model for the validation eras
napi.download_dataset(f"v4.3/meta_model.parquet", round_num=842)
validation["meta_model"] = pd.read_parquet(
    f"v4.3/meta_model.parquet"
)["numerai_meta_model"]

2025-05-09 17:20:43,551 INFO numerapi.utils: starting download
v4.3/meta_model.parquet: 29.0MB [00:31, 926kB/s]                             


In [20]:
# Compute the per-era corr between our predictions and the target values
per_era_corr = validation.groupby("era").apply(
    lambda x: numerai_corr(x[["prediction"]].dropna(), x["target"].dropna())
)

# Compute the per-era mmc between our predictions, the meta model, and the target values
per_era_mmc = validation.dropna().groupby("era").apply(
    lambda x: correlation_contribution(x[["prediction"]], x["meta_model"], x["target"])
)


# Plot the per-era correlation
per_era_corr.plot(
  title="Validation CORR",
  kind="bar",
  figsize=(8, 4),
  xticks=[],
  legend=False,
  snap=False
)
per_era_mmc.plot(
  title="Validation MMC",
  kind="bar",
  figsize=(8, 4),
  xticks=[],
  legend=False,
  snap=False
)

/var/folders/z4/vwhf6x1x6vqdmqksv18l_6j40000gn/T/ipykernel_61470/295801370.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_era_corr = validation.groupby("era").apply(
/var/folders/z4/vwhf6x1x6vqdmqksv18l_6j40000gn/T/ipykernel_61470/295801370.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_era_mmc = validation.dropna().groupby("era").apply(


<Axes: title={'center': 'Validation MMC'}, xlabel='era'>

In [22]:
# Plot the cumulative per-era correlation
per_era_corr.cumsum().plot(
  title="Cumulative Validation CORR",
  kind="line",
  figsize=(8, 4),
  legend=False
)
per_era_mmc.cumsum().plot(
  title="Cumulative Validation MMC",
  kind="line",
  figsize=(8, 4),
  legend=False
)

<Axes: title={'center': 'Cumulative Validation MMC'}, xlabel='era'>

In [23]:
# Compute performance metrics
corr_mean = per_era_corr.mean()
corr_std = per_era_corr.std(ddof=0)
corr_sharpe = corr_mean / corr_std
corr_max_drawdown = (per_era_corr.cumsum().expanding(min_periods=1).max() - per_era_corr.cumsum()).max()

mmc_mean = per_era_mmc.mean()
mmc_std = per_era_mmc.std(ddof=0)
mmc_sharpe = mmc_mean / mmc_std
mmc_max_drawdown = (per_era_mmc.cumsum().expanding(min_periods=1).max() - per_era_mmc.cumsum()).max()

pd.DataFrame({
    "mean": [corr_mean, mmc_mean],
    "std": [corr_std, mmc_std],
    "sharpe": [corr_sharpe, mmc_sharpe],
    "max_drawdown": [corr_max_drawdown, mmc_max_drawdown]
}, index=["CORR", "MMC"]).T

,CORR,MMC
mean,prediction 0.031394 dtype: float64,prediction 0.009268 dtype: float64
std,prediction 0.020704 dtype: float64,prediction 0.013589 dtype: float64
sharpe,prediction 1.516352 dtype: float64,prediction 0.682046 dtype: float64
max_drawdown,prediction 0.029168 dtype: float64,prediction 0.023607 dtype: float64


In [ ]:
# Download latest live features
napi.download_dataset(f"{DATA_VERSION}/live.parquet")

# Load live features
live_features = pd.read_parquet(f"{DATA_VERSION}/live.parquet", columns=feature_set)

# Generate live predictions
live_predictions = model.predict(live_features[feature_set])

# Format submission
pd.Series(live_predictions, index=live_features.index).to_frame("prediction")

In [ ]:
# Define your prediction pipeline as a function
def predict(live_features: pd.DataFrame) -> pd.DataFrame:
    live_predictions = model.predict(live_features[feature_set])
    submission = pd.Series(live_predictions, index=live_features.index)
    return submission.to_frame("prediction")

In [ ]:
# Use the cloudpickle library to serialize your function
import cloudpickle
p = cloudpickle.dumps(predict)
with open("hello_numerai.pkl", "wb") as f:
    f.write(p)

In [25]:
import pickle

# Save the model
with open('lgbm_regressor_model.pkl', 'wb') as file:
    pickle.dump(model, file)

